In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# Cell 4: Data Preprocessing and KNN Model Setup

# Filter users with at least 200 ratings
user_counts = df_ratings['user'].value_counts()
df_ratings_filtered = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]

# Filter books with at least 100 ratings
book_counts = df_ratings_filtered['isbn'].value_counts()
df_ratings_filtered = df_ratings_filtered[df_ratings_filtered['isbn'].isin(book_counts[book_counts >= 100].index)]

# Merge ratings with book titles
df_merged = pd.merge(df_ratings_filtered, df_books, on='isbn', how='inner')

# Create user-book pivot table
pivot_table = df_merged.pivot_table(index='title', columns='user', values='rating').fillna(0)

# Convert to sparse matrix
sparse_matrix = csr_matrix(pivot_table.values)

# Initialize and fit KNN model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)
model_knn.fit(sparse_matrix)

# Store pivot table index for lookup
book_titles = pivot_table.index

In [ ]:
# Cell 5: Function to return recommended books
def get_recommends(book=""):
    # Check if book exists in the dataset
    if book not in book_titles:
        return [book, []]  # Return empty recommendations if book not found
    
    # Get index of the book in the pivot table
    book_idx = np.where(book_titles == book)[0][0]
    
    # Get distances and indices of 6 nearest neighbors (includes the book itself)
    distances, indices = model_knn.kneighbors(sparse_matrix[book_idx], n_neighbors=6)
    
    # Prepare recommendations (exclude the input book, take top 5)
    recommended_books = []
    for i in range(1, 6):  # Start from 1 to skip the input book
        neighbor_idx = indices[0][i]
        neighbor_title = book_titles[neighbor_idx]
        distance = distances[0][i]
        recommended_books.append([neighbor_title, distance])
    
    # Return in the required format: [book_title, [[title1, dist1], [title2, dist2], ...]]
    return [book, recommended_books]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()